<a href="https://colab.research.google.com/github/crisdavid3335/Challenge-MLEngineer/blob/main/Machine_Learning_Engineer_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt



# setting seed for reproducibility
SEED = 34
np.random.seed(SEED)
TARGET_SIZE = 224
PATH_DOGS = '/content/drive/MyDrive/PENTO_IMGS/dogs'

def load_names_images(directory: str) -> dict:
    return {
        breed: [
            file for file in os.listdir(os.path.join(directory, breed))
            if file.lower().endswith(('jpg', 'jpeg', 'png'))
        ]
        for breed in [r for r in os.listdir(directory) if os.path.isdir(os.path.join(directory, r))]
    }

def load_images(data: dict) -> dict:
    return {
        breed: [
            (image.astype('uint8') if image.dtype != np.uint8 else image)
            for image in [
                plt.imread(os.path.join(PATH_DOGS, breed, name))
                for name in tqdm(data[breed], desc=f"Loading images from {breed}", leave=True)
            ]
        ]
        for breed in data
    }


def evaluate_shapes(images) -> None:
  for key in images:
    image_sizes = [img.shape for img in images[key]]
      # Check if all images have the same size
    if len(set(image_sizes)) == 1:
      print(f"All images have the same size: {image_sizes[0]}")
    else:
      print('\n', key, '='*100)
      print(f"Images have varying sizes. Here are some examples: {image_sizes[:5]}")

In [15]:
names = load_names_images(PATH_DOGS)
images = load_images(names)


evaluate_shapes(images)


Loading images from german_shepherd:   0%|          | 0/20 [00:00<?, ?it/s]

Loading images from poodle:   0%|          | 0/20 [00:00<?, ?it/s]

Loading images from golden_retriever:   0%|          | 0/20 [00:00<?, ?it/s]

Loading images from french_bulldog:   0%|          | 0/20 [00:00<?, ?it/s]


 german_shepherd ====================================================================================================
Images have varying sizes. Here are some examples: [(1200, 1920, 3), (1314, 1125, 3), (1412, 1060, 3), (880, 1200, 3), (3008, 2000, 3)]

 poodle ====================================================================================================
Images have varying sizes. Here are some examples: [(1696, 1875, 3), (1500, 1125, 3), (1500, 1500, 3), (1921, 1661, 3), (1879, 2280, 3)]

 golden_retriever ====================================================================================================
Images have varying sizes. Here are some examples: [(1500, 1125, 3), (1200, 1600, 3), (874, 1197, 3), (1194, 1242, 3), (1500, 1125, 3)]

 french_bulldog ====================================================================================================
Images have varying sizes. Here are some examples: [(953, 1027, 3), (1500, 1125, 3), (1500, 1347, 3), (3120, 2956, 3), (1396